In [1]:
!pip install bs4

  Using cached bs4-0.0.1-py3-none-any.whl


In [2]:
!pip install fake_useragent

  Obtaining dependency information for fake_useragent from https://files.pythonhosted.org/packages/56/56/f72e9ca4f9cfb966f489c2b8ea04c67fa8d0cfbb62b1651cb9d6aef110a6/fake_useragent-1.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-resources>=5.0 from https://files.pythonhosted.org/packages/65/6e/09d8816b5cb7a4006ef8ad1717a2703ad9f331dae9717d9f22488a2d6469/importlib_resources-6.1.0-py3-none-any.whl.metadata


In [2]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import nltk
from collections import Counter
import pandas as pd
from time import sleep
from random import randint
import os
import re

In [55]:
import stanza
stanza.download('ru') # download English model
nlp = stanza.Pipeline('ru')

2023-10-18 18:06:07 INFO: Downloading default packages for language: ru (Russian) ...
2023-10-18 18:06:08 INFO: File exists: C:\Users\GL_24_12_2019\stanza_resources\ru\default.zip
2023-10-18 18:06:11 INFO: Finished downloading models and saved to C:\Users\GL_24_12_2019\stanza_resources.
2023-10-18 18:06:11 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-18 18:06:13 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |
| ner       | wikiner            |

2023-10-18 18:06:13 INFO: Using device: cpu
2023-10-18 18:06:13 INFO: Loading: tokenize
2023-10-18 18:06:13 INFO: Loading: pos
2023-10-18 18:06:13 INFO: Loading: lemma
2023-10-1

In [4]:
ua = UserAgent()
headers = {"UserAgent": ua.random}
session = requests.session()

In [3]:
os.mkdir('./better_corpus')

In [3]:
# собираем ссылки на фанфики с одной страницы
def retrieve_links_from_page(search_page_link):
    response = session.get(search_page_link, headers=headers)
    soup = BeautifulSoup(response.text)
    fic_links = []
    links = soup.find_all("a")
    for link in links:
        filt = link.get('href')
        if filt != None:
            if filt.startswith('/s'):
                fic_links.append(filt)
    return fic_links

In [5]:
sp_links = ['https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&search_genrelist1=0&search_genrelist2=0&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3&search_sortby=score', 
            'https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&s=2&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&search_genrelist1=0&search_genrelist2=0&search_sortby=score&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3',
            'https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&s=3&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&search_genrelist1=0&search_genrelist2=0&search_sortby=score&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3',
            'https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&s=4&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&search_genrelist1=0&search_genrelist2=0&search_sortby=score&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3',
            'https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&s=5&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&search_genrelist1=0&search_genrelist2=0&search_sortby=score&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3',
            'https://ru.fanfiktion.net/?a=u&catid=300000263&search=do&s=6&search_title=&search_summary=&search_author=&searchStoryLengthStart=0&searchStoryLengthEnd=101000&search_genrelist1=0&search_genrelist2=0&search_sortby=score&ffcbg-TYPESTORY%5B%5D=13&ffcbg-TYPESTORY%5B%5D=7&ffcbg-TYPESTORY%5B%5D=8&ffcbg-TYPESTORY%5B%5D=4&ffcbg-TYPESTORY%5B%5D=6&ffcbg-TYPESTORY%5B%5D=3&ffcbg-TYPESTORY%5B%5D=2&ffcbg-TYPESTORY%5B%5D=5&ffcbg-TYPESTORY%5B%5D=9&ffcbg-AGEADVICE-RU%5B%5D=1&ffcbg-AGEADVICE-RU%5B%5D=2&ffcbg-AGEADVICE-RU%5B%5D=4&ffcbg-PAIRINGTYPE%5B%5D=1&ffcbg-storystatus%5B%5D=0&ffcbg-storystatus%5B%5D=1&ffcbg-storystatus%5B%5D=2&ffcbg-storystatus%5B%5D=4&ffcbg-storystatus%5B%5D=3']

In [ ]:
# собираем список ссылок на все фанфики на всех страницах 
fic_links = []
for sp_link in sp_links:
    li = retrieve_links_from_page(sp_link)
    fic_links.extend(li)

In [47]:
fic_links

['/s/5d8f687d000012a832a404d4/1/Doktor-dlya-Zolushki',
 '/s/5bfa6b48000016a338dde953/1/Zhenskaya-solidarnost',
 '/s/5ab0ce68000016a3db6c11e/1/Stepen-ne-veroyatnosti',
 '/s/596fc33d00000ab5ff587c9/1/Blagoslovennaya-noch',
 '/s/596fb6e000000ab536985105/1/My-oh-my',
 '/s/596a64b300000ab5291d0537/1/A-thousand-kisses-deep',
 '/s/5963e5e600000ab513bd2d4a/1/White-Light',
 '/s/56cf66a40000123b19501272/1/Fajf-o-klok',
 '/s/563fc38e00000a7d1636f333/1/Zhdem-nastoyashego-Doktora',
 '/s/563b7ef70000116a1c720b07/1/Amy-039-s-Choice',
 '/s/56316af400000a7d1efc32bb/1/Bozhestvennaya-sila',
 '/s/5630148100000a7d205b4ee0/1/Eksperiment',
 '/s/562eb46500000a7d8aeb65e/1/Chasti-pamyati',
 '/s/562d465b00000a7d1af02646/1/Vnutri',
 '/s/562c15e800000a7d21068709/1/Magii-ne-byvaet',
 '/s/562ace3a00000a7d19e1c02c/1/Delo-lyudej',
 '/s/562968da00000a7d25ef91e6/1/Zolotoj-vek',
 '/s/5556703000000a7dd7e4008/1/Odnostrochniki',
 '/s/562823a300000a7d237fecd8/1/Shtorm',
 '/s/5626d66100000a7d12b04563/1/Ego-nastoyashaya-polovi

In [8]:
# добываем текст каждого фанфика, немного его чистим и сохраняем
def get_fic(fic_link):
    full_link = 'https://ru.fanfiktion.net' + fic_link
    response = session.get(full_link, headers=headers)
    soup = BeautifulSoup(response.text)
    name = soup.find('h4', class_ = "huge-font").get_text()
    author = soup.find('a', class_ = "no-wrap").get_text()
    last_page = int(soup.find("span", class_ = "semibold").get_text())
    metadata = f'{{"author": "{author}", "fic_name": "{name}",  "link": "{full_link}"}}'
    link_parts = fic_link.split('/')
    save_path = 'corpus'
    full_path = os.path.join(save_path, link_parts[-1]+".txt")  
    with open(full_path, "w", encoding="UTF-8") as file:
        file.write(metadata)
        for n in range(1, last_page+1):
            link_parts[3] = str(n)
            chapter_link = 'https://ru.fanfiktion.net' + '/'.join(link_parts)
            chapter_response = session.get(chapter_link, headers=headers)
            soup = BeautifulSoup(chapter_response.text)
            chapter_text = soup.find(class_="user-formatted-inner")
            chapter_text = re.sub(r"                                ", "", str(chapter_text))
            clean_html = re.compile('<.*?>') 
            chapter_text = re.sub(clean_html, " ", chapter_text)
            file.write(text)
            sleep(randint(1,3))

In [43]:
lnk = 'https://ru.fanfiktion.net/s/545421bc00000a7d1494942e/1/Sekret-krasnoj-pomady'
rs = session.get(lnk, headers=headers)
sps = BeautifulSoup(rs.text)
txt = sps.find(class_="user-formatted-inner")
txt = re.sub(r"                                ", "", str(txt))
clean_html = re.compile('<.*?>') 
txt = re.sub(clean_html, "", txt)
with open("test_text.txt", "w", encoding = "UTF-8") as f:
    f.write(txt)

In [62]:
for fic_link in fic_links:
    get_fic(fic_link)